In [1]:
!pip install pandas psycopg2-binary
!pip install afinn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 7.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 795.7 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53429 sha256=a43e6917282a4ffd0521a13b16142e54726730d4455ebce8615bc0182fc50133
  Stored in directory: /home/jovyan/.cache/pip/wheels/ee/d3/a0/f9255ebac29886acb1c28b35b37523f6399677fa06be379f25
Successfully built afinn


In [66]:
# setup
import pandas as pd
import numpy as np
import psycopg2 as pg
from afinn import Afinn

engine = pg.connect("dbname='clever' user='clever' host='postgres_clever' port='5432' password='clever'")
pd.set_option('display.max_columns', None)

def get_best_rated_real_estate_companys_per_city_state(city, state):
    city = city.capitalize()
    state = state.capitalize()
    df_company_reviews = pd.read_sql('''select * from "curated.company_reviews" where city = %s and state = %s''', con=engine, params=(city, state,))

    df_company_reviews['company_rating'].replace('nan', np.nan, inplace=True)
    df_company_reviews = df_company_reviews[df_company_reviews['company_rating'].notna()]
    df_company_reviews = df_company_reviews[df_company_reviews['type'].str.lower().str.contains('real estate')]
    
    return df_company_reviews[['company_name', 'type', 'city', 'state', 'company_rating', 'amount_of_reviews']].sort_values(['company_rating', 'amount_of_reviews'], ascending=False)

def get_best_rated_company_per_city_type(city, state, type):
    city = city.capitalize()
    state = state.capitalize()
    type = type.lower()
    df_company_reviews = pd.read_sql('''select * from "curated.company_reviews" where city = %s and state = %s''', con=engine, params=(city, state,))

    df_company_reviews['company_rating'].replace('nan', np.nan, inplace=True)
    df_company_reviews['company_rating'] = pd.to_numeric(df_company_reviews['company_rating'])
    df_company_reviews = df_company_reviews[df_company_reviews['company_rating'].notna()]
    df_company_reviews = df_company_reviews[df_company_reviews['type'].str.lower().str.contains(type)]

    df_company_reviews = df_company_reviews.groupby(['company_name', 'type', 'city', 'state']).agg({'company_rating': 'mean', 'amount_of_reviews': 'mean'}).reset_index()
    
    return df_company_reviews.sort_values(by=['company_rating', 'amount_of_reviews'], ascending=False)

# Using Afinn to assess the sentiment of customer reviews adding to the review rating

def normalize_score(score):
    threshold = 10
    if score is None:
        return None
    normalized = (score + threshold) / (2 * threshold) * 5
    return normalized

def get_best_rated_companys_sentiment_based(city, state):
    afinn = Afinn()
    city = city.capitalize()
    state = state.capitalize()
    
    dataframe = pd.read_sql('''select * from "curated.company_reviews" where city = %s and state = %s''', con=engine, params=(city, state,))

    # Get only reviews with rating
    dataframe = dataframe[dataframe['review_rating'].notnull()]
    dataframe['review_rating'] = pd.to_numeric(dataframe['review_rating'])
    dataframe['sentiment_score'] = dataframe['review_text'].apply(lambda x: afinn.score(x))

    # Normalize the afinn sentiment score to 0 to 5, to be in order with the review rating
    threshold = 10
    dataframe['clipped_score'] = dataframe['sentiment_score'].clip(upper=threshold)
    dataframe['normalized_sentiment'] = dataframe['clipped_score'].apply(normalize_score)
    
    dataframe['final_score'] = (dataframe['normalized_sentiment'] + dataframe['review_rating']) / 2

    company_sentiment = dataframe.groupby(['company_name', 'city', 'state'])['final_score'].mean().reset_index()
    
    return company_sentiment.sort_values('final_score', ascending=False)


df_fmcsa_complaints = pd.read_sql('''select * from "raw.fmcsa_complaints"''', con=engine)
df_fmcsa_companies = pd.read_sql('''select * from "raw.fmcsa_companies"''', con=engine)
df_fmcsa_safer_data = pd.read_sql('''select * from "raw.fmcsa_safer_data"''', con=engine)

def get_moving_companys_with_most_complaints(city):
    dataframe = df_fmcsa_complaints.merge(df_fmcsa_safer_data, how='left', on='usdot_num').merge(df_fmcsa_companies, how='left', on='usdot_num')
    dataframe = dataframe[['usdot_num', 'legal_name', 'city', 'state', 'entity_type', 'complaint_category', 'complaint_year', 'complaint_count']]

    if city:
        dataframe = dataframe[dataframe['city'] == city.upper()]
    
    dataframe = dataframe.groupby(['usdot_num', 'legal_name', 'city', 'state', 'entity_type']
                                 ).agg({'complaint_count': 'sum'}).reset_index().sort_values(by=['complaint_count'], ascending=False)
    
    return dataframe.sort_values('complaint_count', ascending=False)

/tmp/ipykernel_1540/4147273881.py:69: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_fmcsa_complaints = pd.read_sql('''select * from "raw.fmcsa_complaints"''', con=engine)
/tmp/ipykernel_1540/4147273881.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_fmcsa_companies = pd.read_sql('''select * from "raw.fmcsa_companies"''', con=engine)
/tmp/ipykernel_1540/4147273881.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_fmcsa_safer_data = pd.read_sql('''select * from "raw.fmcsa_safer_data"''', con=engine)


In [67]:
df_real_estate = get_best_rated_real_estate_companys_per_city_state('MIAMI', 'florida')
df_real_estate.head(5)

/tmp/ipykernel_1540/4147273881.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_company_reviews = pd.read_sql('''select * from "curated.company_reviews" where city = %s and state = %s''', con=engine, params=(city, state,))


,company_name,type,city,state,company_rating,amount_of_reviews
3159,The Society Real Estate School Florida,Real estate school,Miami,Florida,5.0,232
612,Homeinc- Miami,Real estate agent,Miami,Florida,5.0,222
1281,Blackbook Properties,Real estate agency,Miami,Florida,5.0,184
2364,Yeimy Ortiz PA - Real Estate Broker Associate,Real estate agent,Miami,Florida,5.0,167
403,Richr,Real estate agency,Miami,Florida,5.0,134


In [68]:
df_hotel_dallas = get_best_rated_company_per_city_type('Miami', 'FLORIDA', 'real estate agency')
df_hotel_dallas.head(5)

/tmp/ipykernel_1540/4147273881.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_company_reviews = pd.read_sql('''select * from "curated.company_reviews" where city = %s and state = %s''', con=engine, params=(city, state,))


,company_name,type,city,state,company_rating,amount_of_reviews
69,Blackbook Properties,Real estate agency,Miami,Florida,5.0,184.0
468,Richr,Real estate agency,Miami,Florida,5.0,134.0
170,Elite Real Estate Associates,Real estate agency,Miami,Florida,5.0,118.0
181,Eric Farmelant - The Ibis Group at COMPASS,Real estate agency,Miami,Florida,5.0,103.0
326,Margend Palacios P.A.,Real estate agency,Miami,Florida,5.0,97.0


In [69]:
df_companys_per_sentiment = get_best_rated_companys_sentiment_based('Seattle', 'Washington')
df_companys_per_sentiment.head(5)

/tmp/ipykernel_1540/4147273881.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql('''select * from "curated.company_reviews" where city = %s and state = %s''', con=engine, params=(city, state,))


,company_name,city,state,final_score
0,Assist-2-Sell Buyers & Sellers Home Realty: Ph...,Seattle,Washington,5.000000
10,Get Happy At Home Team,Seattle,Washington,4.822183
4,Chris Kallin - Seattle by Design Group @ Sothe...,Seattle,Washington,4.810484
1,Blue Pacific Real Estate,Seattle,Washington,4.798611
21,Zach McDonald - McDonald Real Estate Group,Seattle,Washington,4.789931


In [70]:
df_most_complainted_companies = get_moving_companys_with_most_complaints('Miami')
df_most_complainted_companies.head(5)

,usdot_num,legal_name,city,state,entity_type,complaint_count
18,3307068,US STANDARD MOVING & STORAGE CORP,MIAMI,FL,BROKER,250
4,2247863,MOVING APT INC,MIAMI,FL,BROKER,152
22,3491781,EASY ROAD MOVING & STORAGE INC,MIAMI,FL,CARRIER,130
10,2886867,ADAMS VAN LINES CORP,MIAMI,FL,BROKER,90
9,2821065,A1A MOVERS LLC,MIAMI,FL,CARRIER,85
